Simple Tag
https://www.pettingzoo.ml/mpe/simple_tag

> This is a predator-prey environment. Good agents (green) are faster and receive a negative reward for being hit by adversaries (red) (-10 for each collision). Adversaries are slower and are rewarded for hitting good agents (+10 for each collision). Obstacles (large black circles) block the way. By default, there is 1 good agent, 3 adversaries and 2 obstacles.

Baseline agent algorithm with experience replay buffer

In [10]:
import os
import time
import enum
import math
import random
import collections
import statistics

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch
import torch.nn
import torch.nn.functional as F

print(torch.version.cuda)

class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self

class TimeDelta(object):
    def __init__(self, delta_time):
        """Convert time difference in seconds to days, hours, minutes, seconds.
        
        Parameters
        ==========
        delta_time : float
            Time difference in seconds.
        """
        self.fractional, seconds = math.modf(delta_time)
        seconds = int(seconds)
        minutes, self.seconds = divmod(seconds, 60)
        hours, self.minutes = divmod(minutes, 60)
        self.days, self.hours = divmod(hours, 24)
    
    def __repr__(self):
        return f"{self.days}-{self.hours:02}:{self.minutes:02}:{self.seconds + self.fractional:02}"

from pettingzoo.mpe import simple_tag_v2
from pettingzoo.utils import random_demo

torch.autograd.set_detect_anomaly(True)
None

10.2


Arguments in instantiate environment.

- num_good: number of good agents
- num_adversaries: number of adversaries
- num_obstacles: number of obstacles
- max_cycles: number of frames (a step for each agent) until game terminates
- continuous_actions: Whether agent action spaces are discrete(default) or continuous

In [11]:
env = simple_tag_v2.env(
    num_good=1,
    num_adversaries=1,
    num_obstacles=0,
    max_cycles=100,
    continuous_actions=False
).unwrapped

### What are the environment parameters?

Adversaries (red) capture non-adversary (green). The map is a 2D grid and everything is initialized in the region [-1, +1]. There doesn't seem to be position clipping for out of bounds, but non-adversary agent are penalized for out of bounds.
Agent's observation is a ndarray vector of concatenated data in the following order:

1. current velocity (2,)
2. current position (2,)
3. relative position (2,) of each landmark
4. relative position (2,) of each other agent
5. velocity (2,) of each other non-adversary agent

When there are 3 adverseries and 3 non-adversaries, then advarsary observation space is 24 dimensional and non-advarsary observation space is 22 dimensional.

The environment is sequential. Agents move one at a time. Agents are either `adversary_*` for adversary or `agent_*` for non-adversary.

Actions:

- 0 is NOP
- 1 is go left
- 2 is go right
- 3 is go down
- 4 is go up

### How to train the agents?

- Use the differental inter-agent learning (DIAL) algorithm.
- Use parameter sharing for DAIL agents. Separate parameter sets for adversary agents and good agents.
- It's not entirely clear the authors accumulate gradients for differentiable communication, but it 

Messages are vectors. Length 4, 5 should work.

Concatenate the messages from all the actors and add them to the message input for the current agent.

The names of agents are: 
adversary_0 adversary_1 adversary_2 agent_0 agent_1 agent_2

In [12]:
env.reset()

In [13]:
def get_agent_counts():
    all_agents = 0
    adversaries = 0
    for agent in env.world.agents:
        all_agents += 1
        adversaries += 1 if agent.adversary else 0
    good_agents = all_agents - adversaries
    return (adversaries, good_agents)

def process_config(config):
    for k, v in config.common.items():
        config.adversary[k] = v
        config.agent[k] = v

n_adversaries, n_good_agents = get_agent_counts()
config = AttrDict(
    discount = 0.99,
    epsilon = 0.2,
    n_episodes=100000,
    batch_size=1,
    update_target_interval=10,
    report_interval=32,
    clip_grad_norm=1.0,
    lr=0.001,
    reward_scale=0.2,
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    common=AttrDict(
        hidden_size=16,
        n_actions=env.action_space(env.agent_selection).n,
    ),
    adversary=AttrDict(
        n_agents=n_adversaries,
        observation_shape=env.observation_space("adversary_0").shape

    ),
    agent=AttrDict(
        n_agents=n_good_agents,
        observation_shape=env.observation_space("agent_0").shape
    )
)
process_config(config)
config

{'discount': 0.99,
 'epsilon': 0.2,
 'n_episodes': 100000,
 'batch_size': 1,
 'update_target_interval': 10,
 'report_interval': 32,
 'clip_grad_norm': 1.0,
 'lr': 0.001,
 'reward_scale': 0.2,
 'device': device(type='cuda'),
 'common': {'hidden_size': 16, 'n_actions': 5},
 'adversary': {'n_agents': 1,
  'observation_shape': (8,),
  'hidden_size': 16,
  'n_actions': 5},
 'agent': {'n_agents': 1,
  'observation_shape': (6,),
  'hidden_size': 16,
  'n_actions': 5}}

In [14]:
class Normalizer(object):
    def __init__(self, env):
        self.n_landmarks = len(env.world.landmarks)
        self.n_allagents = len(env.world.agents)
        self.n_good = sum(map(lambda a: not a.adversary, env.world.agents))
    
    @staticmethod
    def normalize_abs_pos(s):
        """Clip absolute position and scale to [-1, 1]
        s is a scalar or an ndarray of one dimension."""
        return np.clip(s, -1.5, 1.5) / 1.5

    @staticmethod
    def normalize_rel_pos(s):
        """Clip relative position and scale to [-1, 1]
        s is a scalar or an ndarray of one dimension."""
        return np.clip(s, -3, 3) / 3

    def __call__(self, obs):
        # normalize and clip positions
        norm_obs = obs.copy()
        # normalize velocity of current entity
        norm_obs[:2] = norm_obs[:2] / 1.3
        # clip/scale abs. position of current entity
        norm_obs[2:4] = self.normalize_abs_pos(norm_obs[2:4])
        # clip/scale rel. position of other entities
        n_range = self.n_landmarks + self.n_allagents - 1
        for i in range(n_range):
            norm_obs[4 + (2*i):4 + (2*(i + 1))] = self.normalize_rel_pos(
                norm_obs[4 + (2*i):4 + (2*(i + 1))]
            )
        # normalize velocity of other entities
        norm_obs[4 + (2*n_range):] = norm_obs[4 + (2*n_range):] / 1.3
        return norm_obs
    
class RewardsShaper(object):
    def __init__(self, env):
        self.n_landmarks = len(env.world.landmarks)
        # self.n_allagents = len(env.world.agents)
        self.name_to_idx = {agent.name: i for i, agent in enumerate(env.world.agents)}
        self.idx_to_name = {i: agent.name for i, agent in enumerate(env.world.agents)}
        self.goodagent_indices = [
            i for i, agent in enumerate(env.world.agents) if agent.name.startswith("agent")
        ]
        self.adversary_indices = [
            i for i, agent in enumerate(env.world.agents) if agent.name.startswith("adversary")
        ]
        # rdist - distance between adversary-good agent to start computing rewards.
        self.rdist = 2
        # collision_dist - distance between adversary-good agent to count collision.
        #    Based on PettingZoo numbers. 
        self.collision_dist = 0.075 + 0.05

    @staticmethod
    def bound(x):
        if x < 0.9:
            return 0
        if x < 1.0:
            return (x - 0.9) * 10
        return min(np.exp(2 * x - 2), 10)
        
    def __call__(self, agent_name, obs):
        """Compute reshaped rewards from observation for agent given agent name.
        Adversary: start gaining small rewards as it nears good agents.
        
        Good agent: starts gaining small penality as it nears bad agents.
        """
        _obs = obs[4 + (2*self.n_landmarks):]
        agent_idx = self.name_to_idx[agent_name]
        cum_r = 0.
        if agent_name.startswith("agent"):
            # penalty across all adversaries
            for adversary_idx in self.adversary_indices:
                # penalty from distance of adversary; penalty of collision
                other_idx = adversary_idx - 1 if agent_idx < adversary_idx else adversary_idx
                x, y = _obs[2*other_idx:(2*other_idx) + 2]
                d    = math.sqrt(x**2 + y**2)
                if d < self.collision_dist:
                    cum_r -= 10
                else:
                    cum_r -= min(max(1 -  (1/self.rdist)*d, 0), 1)
                
            # penalty from boudary based on PettingZoo
            pos = obs[2:4]
            cum_r -= self.bound(abs(pos[0]))
            cum_r -= self.bound(abs(pos[1]))
        
        elif agent_name.startswith("adversary"):
            # reward across all agents
            for goodagent_idx in self.goodagent_indices:
                # reward from distance to agent; reward of collision
                other_idx = goodagent_idx - 1 if agent_idx < goodagent_idx else goodagent_idx
                x, y = _obs[2*other_idx:(2*other_idx) + 2]
                d    = math.sqrt(x**2 + y**2)
                if d < self.collision_dist:
                    cum_r += 10
                else:
                    cum_r += min(max(1 -  (1/self.rdist)*d, 0), 1)
        
        return cum_r

normalize = Normalizer(env) # norm_obs = normalize(obs)
shapereward = RewardsShaper(env) # reward = shapereward(agent_name, obs)
loss_fn = torch.nn.MSELoss()

In [5]:
class SimpleTagNet(torch.nn.Module):
    """NN Model for the agents. Both good agents and adversaries use this model."""
        
    def __init__(self, config, agent_type):
        super().__init__()
        # self.config = config
        self.device      = config.device
        self.observation_size = math.prod(config[agent_type].observation_shape)
        self.n_actions   = config[agent_type].n_actions
        self.hidden_size = config[agent_type].hidden_size
        self.output_mlp = torch.nn.Sequential(
            torch.nn.Linear(self.observation_size, self.hidden_size),
            torch.nn.ReLU(inplace=True),
#             torch.nn.Linear(self.hidden_size, self.hidden_size * 2),
#             torch.nn.ReLU(inplace=True),
            torch.nn.Linear(self.hidden_size, self.n_actions)
        )
    
    def forward(self, observation):
        """Apply DQN to episode step.
        
        Parameters
        ==========
        observation : ndarray
            The observation vector obtained from the environment.
        
        Returns
        =======
        torch.Tensor
            Vector of Q-value associated with each action.
        """
        observation = torch.tensor(observation, dtype=torch.float, device=self.device)
        Q = self.output_mlp(observation)
        return Q

def choose_action(config, agent_type, Q, is_val=False):
    if not is_val and random.random() < config.epsilon:
        return random.randrange(config[agent_type].n_actions)
    else:
        return torch.argmax(Q).item()

In [6]:
def run_episode(config, adversary_net, should_render=False, is_val=False):
    """Run one episodes.
    
    inputs consist of observation, message (backprop), hidden (backprop) indexed by agent
    outputs consist of action, q-value of action (backprop), reward, done indexed by (step, agent)
    
    Returns
    =======
    AttrDict
        Contains episode metrics:
        - steps : number of steps. All agents take an action at each step.
        - reward : episodic rewards indexed by ('adversary', 'agent').
        - step_records : list of quantities produced indiced by step, ('adversary', 'agent'), agent index.
          Each step record has:
            + observation
            + Q
            + reward
            + done
        - loss : contains episodic losses indexed by ('adversary', 'agent'). To be updated by train_agents()
    """
    episode = AttrDict(
        steps=0,
        reward=AttrDict(adversary=0, agent=0),
        step_records=[],
        loss=AttrDict(adversary=0, agent=0)
    )
    n_agents = config.adversary.n_agents + config.agent.n_agents
    step_record = None
    env.reset()
    for agent_step_idx, agent_name in enumerate(env.agent_iter()):
        if should_render:
            env.render()
        
        if agent_step_idx % n_agents == 0:
            episode.steps += 1
            step_record = AttrDict(adversary={}, agent={})
            episode.step_records.append(step_record)

        obs_curr, reward, done, _ = env.last()
        reward = shapereward(agent_name, obs)
        agent_type, agent_idx = agent_name.split("_")
        agent_idx = int(agent_idx)
        if done:
            step_record[agent_type][agent_idx] = AttrDict(
                observation=obs_curr,
                action=None,
                Q=None,
                reward=reward,
                done=done,
            )
            env.step(None)
            continue
        
        if agent_type == "agent":
            env.step(0)
            step_record[agent_type][agent_idx] = AttrDict(
                observation=obs_curr,
                action=0,
                Q=None,
                reward=reward,
                done=done,
            )
        else:
            # agent_type == "adversary"
            Q_curr = adversary_net(obs_curr)
            action = choose_action(config, agent_type, Q_curr, is_val=is_val)
            env.step(action)
            step_record[agent_type][agent_idx] = AttrDict(
                # inputs to network
                observation=obs_curr,
                # outputs of network / inputs to environment
                action=action,
                # output of environment
                reward=reward,
                done=done,
            )
        episode.reward[agent_type] += reward

    return episode

def train_agents(config, batch, adversary_net, adversary_target_net, adversary_optimizer):
    """Compute loss of episode and update agent
    """
    device = config.device
    discount = torch.tensor(config.discount, dtype=torch.float, device=device)
    for episode in batch:
        for step_idx in reversed(range(episode.steps)):
            for agent_idx in episode.step_records[step_idx].adversary.keys():
                curr_record = episode.step_records[step_idx].adversary[agent_idx]
                if curr_record.done:
                    # agent is done at this step
                    continue
                    
                next_record = episode.step_records[step_idx + 1].adversary[agent_idx]
                r = torch.tensor(next_record.reward, dtype=torch.float, device=device)
                y = None
                if next_record.done:
                    # agent terminates at next step
                    y = r
                else:
                next_o = next_record.observation
#                 print(next_o)
#                 break
                target_Q = adversary_target_net(next_o)
                max_target_Q = torch.max(target_Q)
                y = r + (1-next_record.done)*discount*max_target_Q
#                 y = r + (1-next_record.done)*discount*max_target_Q

                u = curr_record.action
                Q_u = curr_record.Q[u]
#                 adversary_loss = torch.pow(y - Q_u, 2.)
#                 distance_loss = loss_fn(adversary_pos, agent_pos)
#                 adversary_loss = loss_fn(y, Q_u) - 2.*(torch.norm(after_rel_distance) < torch.norm(initial_rel_distance))
                adversary_loss = loss_fn(y, Q_u)
#                 print(distance_loss)
        
                loss = adversary_loss
#                 loss = distance_loss

                adversary_optimizer.zero_grad()
                loss.backward()
                # torch.nn.utils.clip_grad_norm_(adversary_net.parameters(), config.clip_grad_norm)
                adversary_optimizer.step()
                episode.loss.adversary += loss.item()
                # print('here')
                # del adversary_loss

def train(config):
    """
    - Use parameter sharing between agents of the same class.
    - Good agents use one RL model, adversaries use another RL model.
      Train the agents side by side.
    - Separate, disjoint communication channels for two classes of agents,
      maintained by a container to store the messages.
    """
    print("Training the agents...")
    os.makedirs("models/batched-baseline-test", exist_ok=True)
    t0 = time.time()
    device = config.device
    adversary_net = SimpleTagNet(config, "adversary").to(device)
    adversary_target_net = SimpleTagNet(config, "adversary").to(device)
    adversary_target_net.eval()
    print("Created the agent nets.")
    adversary_optimizer = torch.optim.SGD(adversary_net.parameters(), lr=config.lr)
    logger = AttrDict(
        episodic_losses=AttrDict(adversary=[], agent=[]),
        episodic_rewards=AttrDict(adversary=[], agent=[])
    )
    def update_targets():
        adversary_target_net.load_state_dict(adversary_net.state_dict())
    print("Initial update of target nets")
    update_targets()
    
    batch = []
    print("Beginning the episodes...")
    for episode_idx in range(config.n_episodes):
        # Run an episode
        episode = run_episode(config, adversary_net,
                              should_render=episode_idx % 32 == 0 and episode_idx > 1)
        batch.append(episode)
        
        # Train on the episode
        if episode_idx % config.batch_size == 0 and episode_idx > 0:
            train_agents(config, batch, adversary_net,
                         adversary_target_net,
                         adversary_optimizer)
            batch = []
        
        # Logging the reward and los
        logger.episodic_losses.adversary.append(episode.loss.adversary)
        logger.episodic_losses.agent.append(episode.loss.agent)
        logger.episodic_rewards.adversary.append(episode.reward.adversary)
        logger.episodic_rewards.agent.append(episode.reward.agent)

        if episode_idx % config.update_target_interval == 0 and episode_idx > 0:
            print('updating')
            # Update double network
            update_targets()

        if episode_idx % config.report_interval == 0 and episode_idx > 0:
            # Logging
            t1 = time.time()
            tdelta = TimeDelta(round(t1 - t0, 0))
            print(f"on episode {episode_idx} (time taken so far: {tdelta})")
            mean_loss_adversary = statistics.fmean(logger.episodic_losses.adversary[-config.report_interval:])
            mean_reward_adversary = statistics.fmean(logger.episodic_rewards.adversary[-config.report_interval:])
            mean_reward_agent = statistics.fmean(logger.episodic_rewards.agent[-config.report_interval:])
            print(f"     mean loss: adversary {mean_loss_adversary}")
            print(f"     mean reward: adversary {mean_reward_adversary}, agent {mean_reward_agent}")

        if episode_idx % 1000 == 0 and episode_idx > 0:
            print('saving')
            torch.save(
                adversary_net.state_dict(),
                f"models/batched-baseline-test/adversary-net-{episode_idx}.pth"
            )
    
    return adversary_net, logger


In [4]:
list(reversed(range(10)))

[9, 8, 7, 6, 5, 4, 3, 2, 1, 0]

In [ ]:
# train model
adversary_net, logger = train(config)

Training the agents...
Created the agent nets.
Initial update of target nets
Beginning the episodes...
updating
updating
updating
on episode 32 (time taken so far: 0-00:00:5.0)
     mean loss: adversary 0.35503706675294333
     mean reward: adversary 0.125, agent -0.345558202271273
updating
updating
updating
on episode 64 (time taken so far: 0-00:00:7.0)
     mean loss: adversary 0.08816819349732544
     mean reward: adversary 0.0, agent -0.16194511640247625
updating
updating
updating
on episode 96 (time taken so far: 0-00:00:9.0)
     mean loss: adversary 0.07033352024338607
     mean reward: adversary 0.0, agent -0.22132357117150028
updating
updating
updating
on episode 128 (time taken so far: 0-00:00:10.0)
     mean loss: adversary 0.09101418396481989
     mean reward: adversary 0.0, agent -0.08045136824022603
updating
updating
updating
updating
on episode 160 (time taken so far: 0-00:00:12.0)
     mean loss: adversary 0.2021942473111096
     mean reward: adversary 0.0, agent -0.237

updating
updating
on episode 1440 (time taken so far: 0-00:01:24.0)
     mean loss: adversary 0.3252071136571662
     mean reward: adversary 0.125, agent -0.4069540695104835
updating
updating
updating
on episode 1472 (time taken so far: 0-00:01:26.0)
     mean loss: adversary 0.0724004565661279
     mean reward: adversary 0.0, agent -0.13857859154041033
updating
updating
updating
on episode 1504 (time taken so far: 0-00:01:28.0)
     mean loss: adversary 0.19272578152021508
     mean reward: adversary 0.0625, agent -0.1822138042022353
updating
updating
updating
on episode 1536 (time taken so far: 0-00:01:29.0)
     mean loss: adversary 0.34596655412492244
     mean reward: adversary 0.125, agent -0.241171650043186
updating
updating
updating
on episode 1568 (time taken so far: 0-00:01:31.0)
     mean loss: adversary 0.3439967039643785
     mean reward: adversary 0.125, agent -0.37214508694397386
updating
updating
updating
updating
on episode 1600 (time taken so far: 0-00:01:33.0)
     m

updating
updating
updating
on episode 2880 (time taken so far: 0-00:02:46.0)
     mean loss: adversary 0.06471905058078939
     mean reward: adversary 0.0, agent -0.052639889776692775
updating
updating
updating
on episode 2912 (time taken so far: 0-00:02:47.0)
     mean loss: adversary 0.3534287880342908
     mean reward: adversary 0.125, agent -0.4092649071662454
updating
updating
updating
on episode 2944 (time taken so far: 0-00:02:49.0)
     mean loss: adversary 0.20861172791671626
     mean reward: adversary 0.0625, agent -0.14066655887993282
updating
updating
updating
on episode 2976 (time taken so far: 0-00:02:51.0)
     mean loss: adversary 0.38948340528950826
     mean reward: adversary 0.125, agent -0.21321449106837026
updating
updating
updating
saving
on episode 3008 (time taken so far: 0-00:02:53.0)
     mean loss: adversary 0.2054678515706982
     mean reward: adversary 0.0625, agent -0.26158780586309555
updating
updating
updating
updating
on episode 3040 (time taken so far

In [ ]:
adversary_net = SimpleTagNet(config, "adversary").to(config.device)
agent_net = SimpleTagNet(config, "agent").to(config.device)
adversary_net.load_state_dict(torch.load('./models/batched-baseline-test/adversary-net-9000.pth'))
# agent_net.load_state_dict(torch.load('./models/batched-baseline-test/agent-net-9000.pth'))

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 6))
axes = axes.ravel()

axes[0].plot(logger.episodic_losses.adversary[50:], label="adversary")
axes[0].plot(logger.episodic_losses.agent[50:], label="good agent")
axes[0].set_title("loss")
axes[0].legend()

axes[1].plot(logger.episodic_rewards.adversary[50:], label="adversary")
axes[1].plot(logger.episodic_rewards.agent[50:], label="good agent")
axes[1].set_title("reward")
axes[1].legend()
None

In [ ]:
def visualize(config, adversary_net):
    adversary_net.eval()
    with torch.no_grad():
        return run_episode(config, adversary_net, should_render=True, is_val=True)

episode = visualize(config, adversary_net)
print("episode steps", episode.steps)
print("episode rewards", *episode.reward.items())